In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree as et
import pandas as pd
from twilio.rest import Client
import sys
import os

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36",
    'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'en-GB,en-US;q=0.9,en;q=0.8'
}

amazon_urls = ['https://amzn.in/d/fvMIjiK']

# Functions
def get_amazon_price(dom):
    try:
        item_price = dom.xpath('//span[@class="a-offscreen"]/text()')[0]
        item_price = item_price.replace(',', '').replace('₹', '').replace('.00', '')
        return int(item_price)
    except IndexError:
        return None  # Price not found


def get_product_name(dom):
    try:
        name = dom.xpath('//span[@id="productTitle"]/text()')
        return name[0].strip()
    except IndexError:
        return "Unknown Product"


def get_master_price(url, df):
    row = df[df['url'] == url]
    if not row.empty:
        return row.iloc[0]['price']
    return None


# Main script
try:
    df = pd.read_csv('master_Data.csv')
except FileNotFoundError:
    sys.exit("Master data CSV not found. Exiting...")

price_drop_products = []
price_drop_list_url = []

for product_url in amazon_urls:
    response = requests.get(product_url, headers=header)
    soup = BeautifulSoup(response.content, 'html.parser')
    main_dom = et.HTML(str(soup))

    price = get_amazon_price(main_dom)
    product_name = get_product_name(main_dom)
    if price is None:
        print(f"Failed to fetch price for {product_url}. Skipping...")
        continue

    master_price = get_master_price(product_url, df)
    if master_price is None:
        print(f"No master price for {product_name}. Skipping...")
        continue

    if price < master_price:
        change_percentage = round((master_price - price) * 100 / master_price)
        if change_percentage > 10:
            print(f"There is a {change_percentage}% drop in price for {product_name}")
            print(f"Click here to purchase: {product_url}")
            price_drop_products.append(product_name)
            price_drop_list_url.append(product_url)

if not price_drop_products:
    sys.exit("No price drops found.")

# SMS Notification
message_content = "There is a drop in price for {} products. Click to purchase:\n".format(len(price_drop_products))
message_content += "\n".join(price_drop_list_url)

# Twilio credentials from environment variables
account_sid = os.getenv('AC38b15e4b20d8868a51dac50e04d7fdd2c')
auth_token = os.getenv('aa8920f0583311a5c9a21801346e20ca')

if not account_sid or not auth_token:
    sys.exit("Twilio credentials not set. Exiting...")

client = Client(account_sid, auth_token)
message = client.messages.create(
    from_='+14155238886',  
    body=message_content,
    to='7999878432'   
)

print("Notification sent successfully!")


Failed to fetch price for https://amzn.in/d/fvMIjiK. Skipping...


SystemExit: No price drops found.

C:\Users\Vedprakash\AppData\Roaming\Python\Python312\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
